In [1]:
import os
import torch
from yolov3.utils import json_load, json_dump
from yolov3.coco import prepare_data, load_classes, convert_detections_to_coco
from yolov3.evaluate import evaluate
from yolov3.modules import Darknet

In [2]:
coco_data_path = '/media/semyon/Data/Documents/coco/'
coco_val_path = os.path.join(coco_data_path, 'val2017')
coco_annotations_path = os.path.join(coco_data_path, 'annotations')
coco_instances_path = os.path.join(coco_annotations_path, 'instances_val2017.json')

## Prepare annotations

In [3]:
annotations = json_load(coco_instances_path)

In our implementation, we consider only bounding boxes however, COCO uses segmentations and areas are computed for segmentations. We need to recompute areas according to bounding box sizes otherwise we can't reproduce COCO evaluation.

In [4]:
for a in annotations['annotations']:
    w, h = a['bbox'][2:]
    a['area'] = w * h

We also don't use `iscrowd` annotations. Hence, we simply set all `iscrowd` to 0.

In [5]:
for a in annotations['annotations']:
    if 'iscrowd' in a:
        a['iscrowd'] = 0

In [6]:
new_instances_path = os.path.join(coco_annotations_path, 'instances_val2017_bbox.json')
json_dump(annotations, new_instances_path)

## Prepare data

In [4]:
category_id_mapping_path = 'coco_category_id_mapping.json'
category_id_mapping = json_load(category_id_mapping_path)
category_id_mapping = {m['coco']: m['yolo'] - 1 for m in category_id_mapping}

In [5]:
data_path = os.path.join(coco_data_path, 'val2017_prepared')
if not os.path.exists(data_path):
    os.mkdir(data_path)
annotations_path = os.path.join(coco_annotations_path, 'instances_val2017_bbox.json')

In [9]:
prepare_data(coco_val_path, annotations_path, data_path, cat_id_mapping=category_id_mapping)

create output dirs
copy images
done       5000/5000
save images index
convert annotations
indexing annotations
done       5000/5000
save annotations index


## Evaluate YOLOv3

In [6]:
cfg_path = './config/yolov3/yolov3_320.cfg'
wts_path = './config/yolov3/yolov3.weights'
cls_path = './config/yolov3/coco.names'
preds_path = os.path.join(coco_data_path, 'val2017_pred')

In [7]:
classes = load_classes(cls_path)
num_classes = len(classes)

In [8]:
if not os.path.exists(preds_path):
    os.mkdir(preds_path)

In [9]:
model = Darknet(cfg_path)
model.load_weights(wts_path)

In [10]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)data_path = os.path.join(coco_data_path, 'val2017_prepared')

cuda:0


In [11]:
evaluate(model, os.path.join(data_path, 'images'), preds_path, num_classes, device=device, batch_size=4)

read index
evaluation
done       5000/5000
save index


## Covert detections to COCO

In [17]:
category_id_mapping = json_load(category_id_mapping_path)
category_id_mapping = {m['yolo']: m['coco'] for m in category_id_mapping}

In [18]:
coco_detections_path = os.path.join(coco_data_path, 'detections_coco.json')

In [19]:
convert_detections_to_coco(preds_path, coco_detections_path, category_id_mapping=category_id_mapping)